# Импорт модулей и загрузка данных

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import re
import numpy as np
import seaborn as sns

# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import SnowballStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('data/df_out.csv')
df = pd.DataFrame(df)
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,NaN,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00


In [3]:
df.shape

(10000, 20)

# Data Preparation

## Визуализация данных

In [4]:
#sns.histplot(data=df, x=df['price_category'])

In [5]:
# plt.figure(figsize=(16,12), dpi=800)
# sns.scatterplot(data=df_out, x='price_category', y='manufacturer')

In [6]:
# plt.figure(figsize=(16,12), dpi=800)
# sns.scatterplot(data=df_out, x='manufacturer', y='region', hue='price_category')
# plt.xticks(rotation=70);

In [7]:
# plt.figure(figsize=(16,12), dpi=800)

# sns.countplot(data=df, x='fuel', hue='price_category')

In [8]:
# plt.figure(figsize=(16,12), dpi=800)

# sns.countplot(data=df, x='odometer', hue='price_category')

In [9]:
# plt.figure(figsize=(16,12), dpi=800)

# sns.countplot(data=df, x='year', hue='price_category')
# plt.xticks(rotation=60);

## Исследование данных на пропуски и обработка

In [10]:
df_out = df.copy()

In [11]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              10000 non-null  int64  
 1   url             10000 non-null  object 
 2   region          10000 non-null  object 
 3   region_url      10000 non-null  object 
 4   price           10000 non-null  int64  
 5   year            9964 non-null   float64
 6   manufacturer    10000 non-null  object 
 7   model           9872 non-null   object 
 8   fuel            9937 non-null   object 
 9   odometer        10000 non-null  int64  
 10  title_status    9834 non-null   object 
 11  transmission    9955 non-null   object 
 12  image_url       9998 non-null   object 
 13  description     9998 non-null   object 
 14  state           10000 non-null  object 
 15  lat             9902 non-null   float64
 16  long            9902 non-null   float64
 17  posting_date    9998 non-null   

### Обработка колонки 'year'

In [12]:
def extract_and_convert_to_float(description):
    match = re.search(r'\b\d{4}\b', str(description))
    if match:
        return float(match.group())
    else:
        return np.nan

nan_indices = df_out[df_out['year'].isna()].index
df_out.loc[nan_indices, 'year'] = df_out.loc[nan_indices, 'description'].apply(extract_and_convert_to_float)


In [13]:
df_out.loc[df_out['year'].isnull()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
6728,7315259025,https://nh.craigslist.org/cto/d/manchester-201...,new hampshire,https://nh.craigslist.org,3750,NaN,other,NaN,NaN,95674,NaN,NaN,NaN,NaN,nh,NaN,NaN,NaN,low,NaN
9388,7208549803,https://bellingham.craigslist.org/ctd/d/bellin...,bellingham,https://bellingham.craigslist.org,11999,NaN,other,NaN,NaN,95674,NaN,NaN,NaN,NaN,wa,NaN,NaN,NaN,medium,NaN


In [14]:
df_out.dropna(subset=['year'], inplace=True)

In [15]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9998 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9998 non-null   int64  
 1   url             9998 non-null   object 
 2   region          9998 non-null   object 
 3   region_url      9998 non-null   object 
 4   price           9998 non-null   int64  
 5   year            9998 non-null   float64
 6   manufacturer    9998 non-null   object 
 7   model           9872 non-null   object 
 8   fuel            9937 non-null   object 
 9   odometer        9998 non-null   int64  
 10  title_status    9834 non-null   object 
 11  transmission    9955 non-null   object 
 12  image_url       9998 non-null   object 
 13  description     9998 non-null   object 
 14  state           9998 non-null   object 
 15  lat             9902 non-null   float64
 16  long            9902 non-null   float64
 17  posting_date    9998 non-null   object

### Оброботка колонки 'model'

In [16]:
unique_models_in_column = set(df_out['model'].dropna())

def extract_models_from_ads(description):
    match = re.search(r'\b\d{4}\b (\w+)', str(description))
    if match:
        return match.group(1)
    return None

df_out['description_model'] = df_out['description'].apply(extract_models_from_ads)

In [17]:
def fill_nan_models(row):
    if pd.isna(row['model']):
        description_model = row['description_model'] if not pd.isna(row['description_model']) else None
        if description_model in unique_models_in_column:
            return description_model
    return row['model']

df_out['model'] = df_out.apply(fill_nan_models, axis=1)

In [18]:
df_out.drop('description_model', axis=1, inplace=True)

In [19]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9998 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9998 non-null   int64  
 1   url             9998 non-null   object 
 2   region          9998 non-null   object 
 3   region_url      9998 non-null   object 
 4   price           9998 non-null   int64  
 5   year            9998 non-null   float64
 6   manufacturer    9998 non-null   object 
 7   model           9874 non-null   object 
 8   fuel            9937 non-null   object 
 9   odometer        9998 non-null   int64  
 10  title_status    9834 non-null   object 
 11  transmission    9955 non-null   object 
 12  image_url       9998 non-null   object 
 13  description     9998 non-null   object 
 14  state           9998 non-null   object 
 15  lat             9902 non-null   float64
 16  long            9902 non-null   float64
 17  posting_date    9998 non-null   object

In [20]:
def fill_remaining_nan_models(row):
    if pd.isna(row['model']):
        return row['manufacturer']
    return row['model']

df_out['model'] = df_out.apply(fill_remaining_nan_models, axis=1)

In [21]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9998 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9998 non-null   int64  
 1   url             9998 non-null   object 
 2   region          9998 non-null   object 
 3   region_url      9998 non-null   object 
 4   price           9998 non-null   int64  
 5   year            9998 non-null   float64
 6   manufacturer    9998 non-null   object 
 7   model           9998 non-null   object 
 8   fuel            9937 non-null   object 
 9   odometer        9998 non-null   int64  
 10  title_status    9834 non-null   object 
 11  transmission    9955 non-null   object 
 12  image_url       9998 non-null   object 
 13  description     9998 non-null   object 
 14  state           9998 non-null   object 
 15  lat             9902 non-null   float64
 16  long            9902 non-null   float64
 17  posting_date    9998 non-null   object

### Оброботка колонки 'fuel'

In [22]:
fuel_counts = df_out.groupby('model')['fuel'].value_counts()

def most_common_fuel(model):
    if model in fuel_counts:
        return fuel_counts[model].idxmax()[1]
    else:
        return None

df_out['n_fuel'] = df_out['model'].apply(most_common_fuel)

df_out.loc[df_out['fuel'].isna(), 'fuel'] = df_out['n_fuel']

In [23]:
df_out.drop('n_fuel', axis=1, inplace=True)

In [24]:
df_out['fuel'].fillna('other', inplace=True)

In [25]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9998 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9998 non-null   int64  
 1   url             9998 non-null   object 
 2   region          9998 non-null   object 
 3   region_url      9998 non-null   object 
 4   price           9998 non-null   int64  
 5   year            9998 non-null   float64
 6   manufacturer    9998 non-null   object 
 7   model           9998 non-null   object 
 8   fuel            9998 non-null   object 
 9   odometer        9998 non-null   int64  
 10  title_status    9834 non-null   object 
 11  transmission    9955 non-null   object 
 12  image_url       9998 non-null   object 
 13  description     9998 non-null   object 
 14  state           9998 non-null   object 
 15  lat             9902 non-null   float64
 16  long            9902 non-null   float64
 17  posting_date    9998 non-null   object

### Оброботка колонки 'title_status'

In [26]:
df_out['title_status'].fillna('clean', inplace=True)

In [27]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9998 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9998 non-null   int64  
 1   url             9998 non-null   object 
 2   region          9998 non-null   object 
 3   region_url      9998 non-null   object 
 4   price           9998 non-null   int64  
 5   year            9998 non-null   float64
 6   manufacturer    9998 non-null   object 
 7   model           9998 non-null   object 
 8   fuel            9998 non-null   object 
 9   odometer        9998 non-null   int64  
 10  title_status    9998 non-null   object 
 11  transmission    9955 non-null   object 
 12  image_url       9998 non-null   object 
 13  description     9998 non-null   object 
 14  state           9998 non-null   object 
 15  lat             9902 non-null   float64
 16  long            9902 non-null   float64
 17  posting_date    9998 non-null   object

### Оброботка колонки 'transmission'

In [28]:
transmission_counts = df_out.groupby('model')['transmission'].value_counts()

def most_common_transmission(model):
    if model in transmission_counts:
        return transmission_counts[model].idxmax()[1] 
    else:
        return None

df_out['n_transmission'] = df_out['model'].apply(most_common_transmission)

df_out.loc[df['transmission'].isna(), 'transmission'] = df_out['n_transmission']


In [29]:
df_out.drop('n_transmission', axis=1, inplace=True)

In [30]:
df_out['transmission'].fillna('other', inplace=True)

### Оброботка колонки 'lat' и 'long'

In [31]:
region_coordinates = {}

for region in df_out['region'].unique():
    region_data = df_out[df_out['region'] == region].iloc[0]
    region_coordinates[region] = {'lat': region_data['lat'], 'long': region_data['long']}

In [32]:
def fill_na_with_region(row):
    region = row['region']
    if pd.isna(row['lat']) and region in region_coordinates:
        row['lat'] = region_coordinates[region]['lat']
    if pd.isna(row['long']) and region in region_coordinates:
        row['long'] = region_coordinates[region]['long']
    return row

df_out = df_out.apply(fill_na_with_region, axis=1)

In [33]:
df_out.loc[df_out['lat'].isnull()]

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
118,7312692841,https://tricities.craigslist.org/ctd/d/bmw-ser...,tri-cities,https://tricities.craigslist.org,17998,2012.0,bmw,5 series,gas,62665,clean,automatic,https://images.craigslist.org/00909_57Xyuj8AJH...,2012 BMW 5 Series 535i M Sport Heat & Cool Sea...,tn,NaN,NaN,2021-04-26T04:28:35-0400,medium,2021-04-26 08:28:35+00:00
7333,7312541982,https://tricities.craigslist.org/ctd/d/alfa-ro...,tri-cities,https://tricities.craigslist.org,28163,2018.0,alfa-romeo,romeo stelvio,gas,35248,clean,automatic,https://images.craigslist.org/00P0P_2v7cD6oDdL...,2018 Alfa Romeo Stelvio Ti Sport AWD 4x4 SUV N...,tn,NaN,NaN,2021-04-25T17:50:22-0400,high,2021-04-25 21:50:22+00:00


In [34]:
df_out.loc[df_out['region'] == 'tri-cities']

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
118,7312692841,https://tricities.craigslist.org/ctd/d/bmw-ser...,tri-cities,https://tricities.craigslist.org,17998,2012.0,bmw,5 series,gas,62665,clean,automatic,https://images.craigslist.org/00909_57Xyuj8AJH...,2012 BMW 5 Series 535i M Sport Heat & Cool Sea...,tn,NaN,NaN,2021-04-26T04:28:35-0400,medium,2021-04-26 08:28:35+00:00
807,7315738291,https://tricities.craigslist.org/ctd/d/blountv...,tri-cities,https://tricities.craigslist.org,37590,2019.0,ford,f150 supercrew cab xl,gas,8663,clean,other,https://images.craigslist.org/00606_eM2dJpJOQb...,Carvana is the safer way to buy a car During t...,tn,36.530000,-82.320000,2021-05-02T10:41:04-0400,high,2021-05-02 14:41:04+00:00
1176,7303274816,https://tricities.craigslist.org/ctd/d/blountv...,tri-cities,https://tricities.craigslist.org,38590,2019.0,ram,1500 crew cab big horn,gas,28556,clean,other,https://images.craigslist.org/00k0k_5uBzZTLETd...,Carvana is the safer way to buy a car During t...,tn,36.530000,-82.320000,2021-04-07T15:11:07-0400,high,2021-04-07 19:11:07+00:00
1428,7313832415,https://tricities.craigslist.org/ctd/d/piney-f...,tri-cities,https://tricities.craigslist.org,31900,2015.0,ford,f150,gas,86089,clean,automatic,https://images.craigslist.org/00909_d85FdqFNWf...,2015 FORD F150 SUPERCREW Offered by: Legg ...,tn,36.438087,-82.302045,2021-04-28T11:19:16-0400,high,2021-04-28 15:19:16+00:00
1531,7313325374,https://tricities.craigslist.org/ctd/d/blountv...,tri-cities,https://tricities.craigslist.org,25590,2015.0,gmc,sierra 1500 regular cab,other,35290,clean,other,https://images.craigslist.org/00M0M_lsdsAjyaG9...,Carvana is the safer way to buy a car During t...,tn,36.530000,-82.320000,2021-04-27T11:11:29-0400,high,2021-04-27 15:11:29+00:00
1747,7307191301,https://tricities.craigslist.org/ctd/d/piney-f...,tri-cities,https://tricities.craigslist.org,8995,2012.0,ford,escape,gas,136257,clean,automatic,https://images.craigslist.org/00707_k7w05v0FLA...,2012 FORD ESCAPE XLT Offered by: Legg Moto...,tn,36.438087,-82.302045,2021-04-15T11:16:37-0400,low,2021-04-15 15:16:37+00:00
2198,7310959994,https://tricities.craigslist.org/ctd/d/knoxvil...,tri-cities,https://tricities.craigslist.org,20990,2016.0,ford,transit cargo cargo 250,gas,120280,clean,automatic,https://images.craigslist.org/00S0S_hCHebMZYg6...,🚗2016 Ford Transit Cargo Cargo 250 🚗Exterior C...,tn,35.924400,-84.001500,2021-04-22T14:17:51-0400,medium,2021-04-22 18:17:51+00:00
2387,7307968448,https://tricities.craigslist.org/ctd/d/summerf...,tri-cities,https://tricities.craigslist.org,26994,2013.0,ford,f150 limited,gas,132237,clean,automatic,https://images.craigslist.org/01212_72maJMuEwk...,2013 FORD F150 LIMITED CALL Brookbank Auto Exc...,tn,36.224500,-79.890100,2021-04-16T17:54:45-0400,high,2021-04-16 21:54:45+00:00
2926,7314211609,https://tricities.craigslist.org/ctd/d/summerf...,tri-cities,https://tricities.craigslist.org,51902,2016.0,ford,f350 super duty lariat,diesel,99575,clean,automatic,https://images.craigslist.org/00Y0Y_5BlZpJd99d...,2016 FORD F350 SUPER DUTY LARIAT CALL Brookban...,tn,36.224500,-79.890100,2021-04-29T02:59:42-0400,high,2021-04-29 06:59:42+00:00
5064,7314579840,https://tricities.craigslist.org/ctd/d/blountv...,tri-cities,https://tricities.craigslist.org,27990,2018.0,acura,rdx technology pkg sport,gas,30791,clean,other,https://images.craigslist.org/00A0A_RR9ewIm1Mc...,Carvana is the safer way to buy a car During t...,tn,36.530000,-82.320000,2021-04-29T19:01:07-0400,high,2021-04-29 23:01:07+00:00


In [35]:
df_out['lat'].fillna('36.530000', inplace=True)

In [36]:
df_out['long'].fillna('82.320000', inplace=True)

In [37]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9998 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9998 non-null   int64  
 1   url             9998 non-null   object 
 2   region          9998 non-null   object 
 3   region_url      9998 non-null   object 
 4   price           9998 non-null   int64  
 5   year            9998 non-null   float64
 6   manufacturer    9998 non-null   object 
 7   model           9998 non-null   object 
 8   fuel            9998 non-null   object 
 9   odometer        9998 non-null   int64  
 10  title_status    9998 non-null   object 
 11  transmission    9998 non-null   object 
 12  image_url       9998 non-null   object 
 13  description     9998 non-null   object 
 14  state           9998 non-null   object 
 15  lat             9998 non-null   object 
 16  long            9998 non-null   object 
 17  posting_date    9998 non-null   object

### Преобразование типов данных

In [38]:
df_out['year'] = df_out['year'].astype(int)

In [39]:
df_out = df_out.drop(['id'], axis=1)

In [40]:
df_out['manufacturer'] = df_out['manufacturer'].astype('category')

In [41]:
df_out['fuel'] = df_out['fuel'].astype('category')

In [42]:
df_out['title_status'] = df_out['title_status'].astype('category')

In [43]:
df_out['transmission'] = df_out['transmission'].astype('category')

In [44]:
df_out['state'] = df_out['state'].astype('category')

In [45]:
df_out['lat'] = df_out['lat'].astype('float')

In [46]:
df_out['long'] = df_out['long'].astype('float')

In [47]:
df_out['price_category'] = df_out['price_category'].astype('category')

In [48]:
df_out['posting_date'] = pd.to_datetime(df_out['posting_date'], utc=True)

In [49]:
df_out['date'] = pd.to_datetime(df_out['date'])

In [50]:
df_out['region'] = df_out['region'].astype('category')

In [51]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9998 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   url             9998 non-null   object             
 1   region          9998 non-null   category           
 2   region_url      9998 non-null   object             
 3   price           9998 non-null   int64              
 4   year            9998 non-null   int32              
 5   manufacturer    9998 non-null   category           
 6   model           9998 non-null   object             
 7   fuel            9998 non-null   category           
 8   odometer        9998 non-null   int64              
 9   title_status    9998 non-null   category           
 10  transmission    9998 non-null   category           
 11  image_url       9998 non-null   object             
 12  description     9998 non-null   object             
 13  state           9998 non-null   catego

## Работа с выбросами

#### Работа с колонкой 'year'

In [52]:
df_out['year'].describe()

count    9998.000000
mean     2010.877976
std        12.390673
min      1236.000000
25%      2008.000000
50%      2013.000000
75%      2017.000000
max      2022.000000
Name: year, dtype: float64

In [53]:
df_out.loc[df_out['year'] < 2008, 'year'] = 2008
df_out.loc[:, 'year'] = df_out['year'].astype(int)

In [54]:
df_out['year'].describe()

count    9998.000000
mean     2013.090818
std         4.123476
min      2008.000000
25%      2008.000000
50%      2013.000000
75%      2017.000000
max      2022.000000
Name: year, dtype: float64

#### Работа с колонкой 'price'

In [55]:
df_out['price'].describe()

count    9.998000e+03
mean     2.068098e+04
std      1.246353e+05
min      5.000000e+02
25%      7.900000e+03
50%      1.575650e+04
75%      2.799000e+04
max      1.234568e+07
Name: price, dtype: float64

In [56]:
Q1 = df_out['price'].quantile(0.25)
Q3 = df_out['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_out[(df_out['price'] < lower_bound) | (df_out['price'] > upper_bound)]

outliers

,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
21,https://madison.craigslist.org/cto/d/madison-1...,madison,https://madison.craigslist.org,89995,2008,other,oldsmobile 442,gas,111111,clean,automatic,https://images.craigslist.org/00S0S_agk7Nol4CO...,1970 Oldsmobile 442 Convertible 442 Indy Pace ...,wi,43.098400,-89.273400,2021-04-25 15:10:43+00:00,high,2021-04-25 15:10:43+00:00
33,https://kpr.craigslist.org/ctd/d/pasco-2019-fo...,kennewick-pasco-richland,https://kpr.craigslist.org,79999,2019,ford,f450 deisel power stroke,diesel,41759,clean,automatic,https://images.craigslist.org/00D0D_ctOGGthFxQ...,2019 Ford F450 Lariat FX4 Dually 4x4 **Clean C...,wa,46.234838,-119.128015,2021-04-10 02:46:04+00:00,high,2021-04-10 02:46:04+00:00
57,https://inlandempire.craigslist.org/ctd/d/cost...,inland empire,https://inlandempire.craigslist.org,69888,2008,ford,coe,gas,1,clean,automatic,https://images.craigslist.org/00M0M_8jgBCZZuG6...,Awesome Resto-Mod COE Ford Car Hauler. Equippe...,ca,33.680100,-117.908500,2021-04-29 17:49:29+00:00,high,2021-04-29 17:49:29+00:00
96,https://wenatchee.craigslist.org/ctd/d/kittita...,wenatchee,https://wenatchee.craigslist.org,58995,2020,jeep,wrangler,gas,17799,clean,automatic,https://images.craigslist.org/00a0a_chVstv8e4Z...,ＷＩＮＤＹ ＣＨＥＶＲＯＬＥＴ ⭐ No Payments For 90 Days* ⭐...,wa,46.999600,-120.516300,2021-04-23 17:43:27+00:00,high,2021-04-23 17:43:27+00:00
144,https://siouxfalls.craigslist.org/ctd/d/gilber...,sioux falls / SE SD,https://siouxfalls.craigslist.org,59900,2014,ford,f550 xl,diesel,156000,clean,automatic,https://images.craigslist.org/00x0x_httQ0BwNkq...,**FOR SALE** 2014 Ford F550 Super Duty 45' Al...,sd,42.117274,-88.436783,2021-04-12 15:34:12+00:00,high,2021-04-12 15:34:12+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9926,https://tulsa.craigslist.org/cto/d/tulsa-2017-...,tulsa,https://tulsa.craigslist.org,65000,2017,rover,rover,gas,47000,clean,automatic,https://images.craigslist.org/00s0s_iGpmplDL99...,"2017 Range Rover HSE, supercharged V8, silver ...",ok,36.126200,-95.940800,2021-05-03 21:37:36+00:00,high,2021-05-03 21:37:36+00:00
9955,https://palmsprings.craigslist.org/ctd/d/fonta...,palm springs,https://palmsprings.craigslist.org,68995,2016,ram,2500,diesel,61130,clean,automatic,https://images.craigslist.org/00p0p_5PdfAkd0AM...,2016 Ram 2500 Laramie Offered by: R&B Auto...,ca,34.107220,-117.455310,2021-05-01 20:40:57+00:00,high,2021-05-01 20:40:57+00:00
9971,https://spokane.craigslist.org/ctd/d/puyallup-...,spokane / coeur d'alene,https://spokane.craigslist.org,86977,2017,ford,f450 super duty crew cab,diesel,16806,clean,automatic,https://images.craigslist.org/00H0H_5P0NdUIgHi...,2017 *** Ford F450 Super Duty Crew Cab Platinu...,id,47.197304,-122.291776,2021-04-29 17:03:19+00:00,high,2021-04-29 17:03:19+00:00
9978,https://holland.craigslist.org/ctd/d/otsego-20...,holland,https://holland.craigslist.org,61022,2020,gmc,sierra 1500,gas,5894,clean,automatic,https://images.craigslist.org/00i0i_64FIhCkyTb...,"2020 GMC Sierra 1500 AT4 -- $61,022 ☎ Call ...",mi,42.448987,-85.661948,2021-04-15 15:05:47+00:00,high,2021-04-15 15:05:47+00:00


In [57]:
column_name = 'price'

top_10_largest = df_out.nlargest(50, column_name)

top_10_smallest = df_out.nsmallest(10, column_name)

top_10_largest

,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
6331,https://akroncanton.craigslist.org/ctd/d/canto...,akron / canton,https://akroncanton.craigslist.org,12345678,2019,chevrolet,chevrolet,gas,100000,clean,automatic,https://images.craigslist.org/00F0F_g6k0zWbump...,"WE HAVE MOVED TO 2517 FULTON DRIVE NW CANTON, ...",oh,40.846500,-81.440800,2021-04-05 16:26:23+00:00,high,2021-04-05 16:26:23+00:00
9792,https://delaware.craigslist.org/cto/d/delaware...,delaware,https://delaware.craigslist.org,1000000,2022,other,Any make any model,other,283466,clean,other,https://images.craigslist.org/00S0S_k3A68PVXRC...,We come to you and give you an easy quick tran...,de,39.572500,-75.595700,2021-04-11 17:51:42+00:00,high,2021-04-11 17:51:42+00:00
5662,https://winchester.craigslist.org/ctd/d/manass...,winchester,https://winchester.craigslist.org,304995,2021,porsche,911,gas,22,clean,automatic,https://images.craigslist.org/00r0r_gh2TnrNdSu...,2021 PORSCHE 911 Turbo Offered by: Insider...,va,38.762669,-77.461754,2021-04-28 19:49:25+00:00,high,2021-04-28 19:49:25+00:00
292,https://tucson.craigslist.org/ctd/d/tucson-201...,tucson,https://tucson.craigslist.org,239995,2017,ferrari,488 gtb,gas,6012,clean,automatic,https://images.craigslist.org/00t0t_dN5Wr8nfOx...,2017 Ferrari 488 GTB Coupe2017 Ferrari 488 GTB...,az,32.236075,-110.878886,2021-04-22 16:38:59+00:00,high,2021-04-22 16:38:59+00:00
6121,https://daytona.craigslist.org/ctd/d/orlando-2...,daytona beach,https://daytona.craigslist.org,197999,2015,other,Lamborghini Huracan,gas,33875,clean,other,https://images.craigslist.org/01717_55Ai9omMq5...,CREDIT AMNESTY! SE HABLA ESPANOL !!OPEN SUNDA...,fl,28.466088,-81.396429,2021-05-01 15:48:44+00:00,high,2021-05-01 15:48:44+00:00
9164,https://desmoines.craigslist.org/ctd/d/portlan...,des moines,https://desmoines.craigslist.org,163540,2020,mercedes-benz,mercedes-benz,other,6318,clean,automatic,https://images.craigslist.org/00y0y_i26NvL7pZC...,2020 *** Mercedes-Benz G-Class G63 AMG*** ...,ia,45.494265,-122.579192,2021-04-14 16:26:24+00:00,high,2021-04-14 16:26:24+00:00
8840,https://austin.craigslist.org/cto/d/austin-sup...,austin,https://austin.craigslist.org,150000,2008,other,Peterbilt 378,diesel,1234,clean,manual,https://images.craigslist.org/00v0v_ejAvqrK3iP...,Selling a Wonderful Super Dump! 2006 Peterbilt...,tx,30.364900,-97.682700,2021-04-28 02:01:28+00:00,high,2021-04-28 02:01:28+00:00
4245,https://portland.craigslist.org/clc/ctd/d/glad...,portland,https://portland.craigslist.org,149990,2015,mercedes-benz,sprinter 3500 airs,diesel,49291,clean,automatic,https://images.craigslist.org/00B0B_79v5I132cL...,2015 Mercedes-Benz Sprinter 3500 Airstream Int...,or,45.382346,-122.605132,2021-05-03 18:04:09+00:00,high,2021-05-03 18:04:09+00:00
9024,https://sandiego.craigslist.org/csd/ctd/d/dall...,san diego,https://sandiego.craigslist.org,139950,2017,other,Mclaren 570GT W/ Upgrades,gas,13000,clean,automatic,https://images.craigslist.org/00N0N_2egaCVi4pq...,SPECIALTYEXOTICS.COMCALL/TXT 469-215-1902FREE ...,ca,32.790400,-96.804400,2021-04-30 15:20:20+00:00,high,2021-04-30 15:20:20+00:00
6566,https://orlando.craigslist.org/ctd/d/orlando-2...,orlando,https://orlando.craigslist.org,129999,2018,mercedes-benz,maybach s560,gas,6000,clean,automatic,https://images.craigslist.org/00u0u_fKyDNpMeQR...,"Serious/ready buyers, please call or text @ (4...",fl,28.463700,-81.394800,2021-04-25 16:58:01+00:00,high,2021-04-25 16:58:01+00:00


In [58]:
top_10_smallest

,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
329,https://bham.craigslist.org/ctd/d/clanton-we-a...,birmingham,https://bham.craigslist.org,500,2021,other,SPECIAL FINANCE PROGRAM 2020,other,1400,clean,other,https://images.craigslist.org/00n0n_5VG28y2JRQ...,WE ARE GOING THROUGH A VERY UNCERTAIN TIME WIT...,al,32.922900,-86.545000,2021-05-03 13:44:39+00:00,low,2021-05-03 13:44:39+00:00
683,https://binghamton.craigslist.org/ctd/d/orange...,binghamton,https://binghamton.craigslist.org,500,2015,honda,accord,gas,56000,clean,automatic,https://images.craigslist.org/00X0X_j1PWPLM8u9...,500DOWNNOW.COM!!!!!!!!🌞 🌞 🌞 America's Largest ...,ny,40.769200,-74.235500,2021-04-16 18:01:34+00:00,low,2021-04-16 18:01:34+00:00
849,https://sanantonio.craigslist.org/cto/d/jbsa-f...,san antonio,https://sanantonio.craigslist.org,500,2008,chevrolet,cobalt lt,gas,180000,clean,automatic,https://images.craigslist.org/00j0j_a3WpNEkmuA...,500 OBO need gone by tonight\today...no quest...,tx,29.448500,-98.428500,2021-04-23 08:41:24+00:00,low,2021-04-23 08:41:24+00:00
878,https://palmsprings.craigslist.org/ctd/d/jurup...,palm springs,https://palmsprings.craigslist.org,500,2013,nissan,altima,gas,130000,salvage,automatic,https://images.craigslist.org/00V0V_h6SQcseixB...,Call* 909 333 8413 Or 323 713 7682 Follow us o...,ca,34.003300,-117.445000,2021-05-01 01:08:52+00:00,low,2021-05-01 01:08:52+00:00
994,https://cleveland.craigslist.org/ctd/d/clevela...,cleveland,https://cleveland.craigslist.org,500,2011,jeep,grand cherokee,gas,80125,clean,automatic,https://images.craigslist.org/00k0k_BQxx1STYRU...,CITYWIDEAUTOMARTS.COM - 6325 BROADWAY AVE. 441...,oh,41.463159,-81.646343,2021-04-15 14:38:10+00:00,low,2021-04-15 14:38:10+00:00
1194,https://minneapolis.craigslist.org/ank/ctd/d/l...,minneapolis / st paul,https://minneapolis.craigslist.org,500,2008,chevrolet,equinox,gas,100000,clean,automatic,https://images.craigslist.org/00W0W_l1R4dela7J...,Updated 5-03-21 *THE FOLLOWING LIST OF CARS A...,mn,45.925259,-94.346037,2021-05-04 17:01:14+00:00,low,2021-05-04 17:01:14+00:00
1494,https://phoenix.craigslist.org/evl/ctd/d/mesa-...,phoenix,https://phoenix.craigslist.org,500,2015,other,Flexible Down Payments,gas,500,clean,automatic,https://images.craigslist.org/00w0w_jTidXLPAgL...,WE ARE STRONG 💪 AT GETTING YOU APPROVED AND D...,az,33.407478,-111.868393,2021-05-04 18:23:28+00:00,low,2021-05-04 18:23:28+00:00
1995,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,500,2010,chevrolet,malibu,gas,123456,clean,automatic,https://images.craigslist.org/00N0N_9ZOafFcSxn...,865-275-6171--CALL STEVE 24/7!! SIMPLE EASY F...,tn,36.048228,-83.952543,2021-05-02 19:07:15+00:00,low,2021-05-02 19:07:15+00:00
2064,https://houston.craigslist.org/ctd/d/houston-l...,houston,https://houston.craigslist.org,500,2015,kia,optima gdi,gas,128000,clean,automatic,https://images.craigslist.org/00r0r_ibMupxw73B...,Fill your online application on our website: s...,tx,29.790280,-95.404716,2021-04-29 19:15:35+00:00,low,2021-04-29 19:15:35+00:00
2244,https://nh.craigslist.org/ctd/d/salem-bad-cred...,new hampshire,https://nh.craigslist.org,500,2012,other,PROGRAMS as little as $500 DOWN!,gas,50000,clean,automatic,https://images.craigslist.org/00606_hHuL9bNLUg...,"👉 BAD Credit, NO Credit - OKAY! We FINANCE ANY...",nh,42.756379,-71.210435,2021-04-29 00:10:37+00:00,low,2021-04-29 00:10:37+00:00


In [59]:
df_out[df_out['manufacturer'] == 'chevrolet']['price'].mean()

30188.195668986853

In [60]:
df_out['price'] = df_out['price'].replace({12345678: 30200})

In [61]:
df_out = df_out.drop(index=9792)

In [62]:
df_out.describe()

,price,year,odometer,lat,long
count,9997.000000,9997.000000,9997.000000,9997.000000,9997.000000
mean,19351.097929,2013.089927,93304.643993,38.578668,-94.015225
std,15530.083441,4.122719,63162.018542,5.831533,18.271292
min,500.000000,2008.000000,0.000000,-67.144243,-158.069300
25%,7900.000000,2008.000000,39142.000000,34.832550,-108.572700
50%,15750.000000,2013.000000,89000.000000,39.260000,-87.947800
75%,27990.000000,2017.000000,136875.000000,42.402740,-80.721251
max,304995.000000,2022.000000,283466.000000,64.947500,173.885502


In [63]:
df_out['odometer'].describe()

count      9997.000000
mean      93304.643993
std       63162.018542
min           0.000000
25%       39142.000000
50%       89000.000000
75%      136875.000000
max      283466.000000
Name: odometer, dtype: float64

## Преоброзавание колонки "description"

In [64]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [65]:
# def preprocess_text(text):
#     tokens = word_tokenize(text)
    
#     tokens_lower = [token.lower() for token in tokens]
    
#     stop_words = set(stopwords.words("english"))
#     filtered_tokens = [token for token in tokens_lower if token not in stop_words]
    
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
#     return lemmatized_tokens

In [66]:
# df_out['processed_description'] = df_out['description'].apply(preprocess_text)

In [67]:
# df_out['processed_description']

## Преоброзавание колонки "model"

In [68]:
# df_out['processed_moedl'] = df_out['model'].apply(preprocess_text)

In [69]:
# df_out['processed_moedl']

# Feature engineering

## Подготовка категориальных переменных с помощью OneHotEncoder

In [70]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9997 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   url             9997 non-null   object             
 1   region          9997 non-null   category           
 2   region_url      9997 non-null   object             
 3   price           9997 non-null   int64              
 4   year            9997 non-null   int32              
 5   manufacturer    9997 non-null   category           
 6   model           9997 non-null   object             
 7   fuel            9997 non-null   category           
 8   odometer        9997 non-null   int64              
 9   title_status    9997 non-null   category           
 10  transmission    9997 non-null   category           
 11  image_url       9997 non-null   object             
 12  description     9997 non-null   object             
 13  state           9997 non-null   catego

In [71]:
df_out['short_model'] = df_out['model'].apply(lambda x: x.split()[0] if isinstance(x, str) else x)
df_out['short_model'].value_counts()

short_model
silverado    416
wrangler     210
1500         206
grand        204
f-150        204
            ... 
defender       1
slc            1
i30            1
isf            1
durangon       1
Name: count, Length: 1013, dtype: int64

In [72]:
categorical_columns = ['manufacturer', 
                       'fuel', 
                       'transmission', 'state',
                       'short_model']


encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_categorical = encoder.fit_transform(df_out[categorical_columns])

In [73]:
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_columns))
encoded_categorical_df.index = df_out.index

df_out_clean = pd.concat([df_out, encoded_categorical_df], axis=1)

df_out_clean.drop(categorical_columns, axis=1, inplace=True)

In [74]:
df_out_clean

,url,region,region_url,price,year,model,odometer,title_status,image_url,description,...,short_model_xts,short_model_xv,short_model_yaris,short_model_yukon,short_model_z,short_model_z3,short_model_z4,short_model_zephyr,short_model_zx2,short_model_♿
0,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,2500 crew cab big horn,27442,clean,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,explorer 4wd 4dr xlt,60023,clean,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,golf r hatchback,14048,clean,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,rav4,117291,clean,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021,altima,8000,clean,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,2008,rav4,150000,clean,https://images.craigslist.org/00n0n_aaEBqmzz5Z...,Transmission:Automatic Exterior Color:WHITE In...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,2008,wrangler,113573,clean,https://images.craigslist.org/00000_9ZNVdXaiei...,2008 Jeep Wrangler X Offered by: R&L Auto -- ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,2011,a3 2.0t premium plus pzev,150184,clean,https://images.craigslist.org/00X0X_k5JiZDDyqM...,2011 Audi A3 2.0T Premium Plus PZEV Wagon2011 ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,2015,cayenne,61943,clean,https://images.craigslist.org/00O0O_gBihGEQxYT...,"2015 Porsche Cayenne AWD 4dr S E-Hybrid - $31,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Подготовка категориальных переменных с помощью TfidfVectorizer

In [75]:
# categorical_columns_n = ['model', 'description']

# tfidf_model = TfidfVectorizer()
# tfidf_model_matrix = tfidf_model.fit_transform(df_out_clean['model'])

# tfidf_description = TfidfVectorizer()
# tfidf_description_matrix = tfidf_description.fit_transform(df_out_clean['description'])

In [76]:
# tfidf_model_df = pd.DataFrame.sparse.from_spmatrix(tfidf_model_matrix, 
#                                                    index=df_out_clean.index, 
#                                                    columns=tfidf_model.get_feature_names_out(['model']))


# tfidf_description_df = pd.DataFrame.sparse.from_spmatrix(tfidf_description_matrix, 
#                                                          index=df_out_clean.index, 
#                                                          columns=tfidf_description.get_feature_names_out(['description']))

In [77]:
# df_out_clean = pd.concat([df_out_clean, tfidf_model_df], axis=1)
# df_out_clean = pd.concat([df_out_clean, tfidf_description_df], axis=1)


In [78]:
# df_out_clean

## Стандартизация и нормализация переменных

In [79]:
df_out_clean['month'] = df_out_clean['date'].dt.month
df_out_clean['dayofweek'] = df_out_clean['date'].dt.dayofweek
df_out_clean['diff_years'] = df_out_clean['date'].dt.year - df_out_clean['year']

In [80]:
numeric_columns = ['year', 'odometer', 'diff_years']

scaler_standard = StandardScaler()
df_out_clean[numeric_columns] = scaler_standard.fit_transform(df_out_clean[numeric_columns])

scaler_minmax = MinMaxScaler()
df_out_clean[numeric_columns] = scaler_minmax.fit_transform(df_out_clean[numeric_columns])

In [81]:
df_out_clean

,url,region,region_url,price,year,model,odometer,title_status,image_url,description,...,short_model_yukon,short_model_z,short_model_z3,short_model_z4,short_model_zephyr,short_model_zx2,short_model_♿,month,dayofweek,diff_years
0,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,0.857143,2500 crew cab big horn,0.096809,clean,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,5,0.142857
1,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,0.571429,explorer 4wd 4dr xlt,0.211747,clean,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,0.428571
2,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,0.642857,golf r hatchback,0.049558,clean,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,2,0.357143
3,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,0.357143,rav4,0.413774,clean,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,5,0.642857
4,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,0.928571,altima,0.028222,clean,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,0.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,https://chautauqua.craigslist.org/ctd/d/falcon...,chautauqua,https://chautauqua.craigslist.org,4495,0.000000,rav4,0.529164,clean,https://images.craigslist.org/00n0n_aaEBqmzz5Z...,Transmission:Automatic Exterior Color:WHITE In...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,5,1.000000
9996,https://binghamton.craigslist.org/ctd/d/roches...,binghamton,https://binghamton.craigslist.org,14495,0.000000,wrangler,0.400658,clean,https://images.craigslist.org/00000_9ZNVdXaiei...,2008 Jeep Wrangler X Offered by: R&L Auto -- ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,1.000000
9997,https://salem.craigslist.org/ctd/d/salem-2011-...,salem,https://salem.craigslist.org,8995,0.214286,a3 2.0t premium plus pzev,0.529813,clean,https://images.craigslist.org/00X0X_k5JiZDDyqM...,2011 Audi A3 2.0T Premium Plus PZEV Wagon2011 ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,3,0.785714
9998,https://madison.craigslist.org/ctd/d/madison-2...,madison,https://madison.craigslist.org,31900,0.500000,cayenne,0.218520,clean,https://images.craigslist.org/00O0O_gBihGEQxYT...,"2015 Porsche Cayenne AWD 4dr S E-Hybrid - $31,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,2,0.500000


In [82]:
drop_columns = ['url', 'region', 'region_url', 'title_status', 
                'image_url', 'description', 'lat', 'long',
                'posting_date', 'date', 'model']

df_out_clean = df_out_clean.drop(drop_columns, axis=1)

## Сохранение дата сета

In [83]:
df_out_clean.to_csv('data/df_out_clean.csv', index=False)

# Modelling

## Формирование датасета для обучения и инициализация переменных

In [106]:
df_out_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9997 entries, 0 to 9999
Columns: 1116 entries, price to diff_years
dtypes: category(1), float64(1112), int32(2), int64(1)
memory usage: 85.1 MB


In [110]:
df_out_clean_copy = df_out_clean.copy()

float_columns = df_out_clean_copy.select_dtypes(include='float64').columns
df_out_clean_copy[float_columns] = df_out_clean_copy[float_columns].astype('float32')

In [111]:
X = df_out_clean_copy.drop(['price_category', 'price', 'diff_years'], axis=1)
y = df_out_clean_copy['price_category']

## Разделение данных на тренировочную и тестовую выборки

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Инициализация моделей

In [113]:
logreg = LogisticRegression(random_state=42)
rfc = RandomForestClassifier(random_state=42)
mlp = MLPClassifier(random_state=42)

## Обучение моделей

In [ ]:
for model in [logreg, rfc, mlp]:
    cv_score = cross_val_score(estimator=model, X=X, y=y, cv=5, error_score='raise')
    print(cv_score.mean(), cv_score.std())

D:\conda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\conda\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\c

0.7843348674337168 0.00784736524612056
0.7804345172586293 0.0016818060305536175


D:\conda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
D:\conda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
D:\conda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
D:\conda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Тюнинг параметров

In [ ]:
from sklearn.model_selection import GridSearchCV

n_estimators = [300,500,700]
max_features = ['sqrt']
max_depth = [2,3,7,11,15]
min_samples_split = [2,3,4,22,23,24]
min_samples_leaf = [2,3,4,5,6,7]
bootstrap = [False]

In [ ]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rfc_2 = RandomForestClassifier(random_state=42)
gs = GridSearchCV(rfc_2, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_train, y_train)

In [ ]:
gs.best_params_

In [ ]:
gs_pred = gs.predict(X_test)

In [ ]:
accuracy_score(y_test, gs_pred)

In [ ]:
confusion_matrix(y_test, gs_pred)

In [89]:
rfc_8 = RandomForestClassifier(random_state=42, max_features='log2', 
                               min_samples_leaf=1, min_samples_split=2, 
                               n_jobs=-1, n_estimators=1600)

In [90]:
rfc_8.fit(X_train, y_train)

RandomForestClassifier(max_features='log2', n_estimators=1600, n_jobs=-1,
                       random_state=42, warm_start=True)

In [91]:
rfc_8_pred = rfc_8.predict(X_test)

In [92]:
accuracy_score(y_test, rfc_8_pred)

0.7783333333333333

In [93]:
confusion_matrix(y_test, rfc_8_pred)

array([[931,  29, 128],
       [ 27, 773, 140],
       [163, 178, 631]], dtype=int64)

In [102]:
logreg_1 = LogisticRegression(random_state=42, penalty='l2', 
                              C=0.1, solver='lbfgs', max_iter=1500)

In [103]:
logreg_1.fit(X_train, y_train)

LogisticRegression(C=0.1, max_iter=1000, random_state=42)

In [104]:
logreg_2_pred = logreg_1.predict(X_test)

In [105]:
accuracy_score(y_test, logreg_2_pred)

0.755

In [98]:
confusion_matrix(y_test, logreg_1_pred)

array([[896,  31, 161],
       [ 26, 800, 114],
       [138, 172, 662]], dtype=int64)

In [99]:
logreg_train = logreg_1.predict(X_train)

In [100]:
accuracy_score(y_train, logreg_train)

0.8419322566814349

In [101]:
confusion_matrix(y_train, logreg_train)

array([[2145,   50,  211],
       [  62, 1983,  239],
       [ 203,  341, 1763]], dtype=int64)

# Results

Наилучшие показатели по всем проверкам выдает модель логистической регрессии. Нужно будет позже заново пройтись по данным и поработать над ними.

In [ ]:
logreg_fin_model = LogisticRegression(random_state=42, penalty='l2', 
                              C=1.0, solver='lbfgs', max_iter=1000)

In [ ]:
logreg_fin_model.fit(X, y)

In [ ]:
with open('logreg_fin_model.pkl', 'wb') as file:
    pickle.dump(logreg_fin_model, file)

In [ ]:
from sklearn.feature_selection import mutual_info_classif

mutual_info = mutual_info_classif(X, y)
mutual_info_scores = pd.Series(mutual_info, index=X.columns, name='Mutual Information')
mutual_info_scores = mutual_info_scores.sort_values(ascending=False)

In [ ]:
print(mutual_info_scores)